In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#Importar el Data Set
dataSet = pd.read_csv("50_Startups.csv")
X = dataSet.iloc[:, :-1]
y = dataSet.iloc[:, 4].values

In [4]:
X.head()

,R&D Spend,Administration,Marketing Spend,State
0,165349.20,136897.80,471784.10,New York
1,162597.70,151377.59,443898.53,California
2,153441.51,101145.55,407934.54,Florida
3,144372.41,118671.85,383199.62,New York
4,142107.34,91391.77,366168.42,Florida


In [5]:
#Codificar datos categóricos
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(), [3])],
    remainder='passthrough'
)
X = ct.fit_transform(X)

In [6]:
#Convertirlo en DataFrame para verlo con nombres de columnas
pd.DataFrame(X).head()

,0,1,2,3,4,5
0,0.0,0.0,1.0,165349.20,136897.80,471784.10
1,1.0,0.0,0.0,162597.70,151377.59,443898.53
2,0.0,1.0,0.0,153441.51,101145.55,407934.54
3,0.0,0.0,1.0,144372.41,118671.85,383199.62
4,0.0,1.0,0.0,142107.34,91391.77,366168.42


In [7]:
#Eliminar la primera columna Dummy para evitar colinealidad
X = X[:, 1:]


In [8]:
pd.DataFrame(X).head()

,0,1,2,3,4
0,0.0,1.0,165349.20,136897.80,471784.10
1,0.0,0.0,162597.70,151377.59,443898.53
2,1.0,0.0,153441.51,101145.55,407934.54
3,0.0,1.0,144372.41,118671.85,383199.62
4,1.0,0.0,142107.34,91391.77,366168.42


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [15]:
print("X_train.shape = ", X_train.shape, " X.shape = ", X.shape)

X_train.shape =  (40, 5)  X.shape =  (50, 5)


In [18]:
#Ajustar el modelo RLM con Conjunto de Entrenamiento
from sklearn.linear_model import LinearRegression
regression = LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [22]:
#Visualizar el modelo de RLM que acabo de crear
from IPython.display import Markdown, display
b0 = regression.intercept_
coefs = regression.coef_

#Crear ecuacion dinámica
equation = f"y = {b0:.4f}"
for i, b in enumerate(coefs):
    equation += f" + {b:.4f}·x{i+1}"

display(Markdown(f"**{equation}**"))

**y = 42554.1676 + -959.2842·x1 + 699.3691·x2 + 0.7735·x3 + 0.0329·x4 + 0.0366·x5**

In [23]:
#Prediccion de resultados en Conjunto de Testing
y_pred = regression.predict(X_test)

In [25]:
y_pred

array([103015.20159797, 132582.27760815, 132447.73845175,  71976.09851258,
       178537.48221055, 116161.24230165,  67851.69209676,  98791.73374688,
       113969.43533012, 167921.0656955 ])

In [42]:
y_test

array([103282.38, 144259.4 , 146121.95,  77798.83, 191050.39, 105008.31,
        81229.06,  97483.56, 110352.25, 166187.94])

In [39]:
#Construir el modelo óptimo de RLM usando Eliminación Hacia Atrás
import statsmodels.api as sm
#Se añade una columna de unos para que esta se multiplique con el "intercepto"; o sea, gracias a esto, damos lugar a la constante
X = np.append(arr = np.ones((50,1)).astype(int), values = X, axis = 1)
SL = 0.05

In [40]:
X_opt = X[:, [0,1,2,3,4,5]]
regression_OLS = sm.OLS(endog = y, exog = X_opt).fit()

In [46]:
regression_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     272.4
Date:                Tue, 11 Nov 2025   Prob (F-statistic):           2.76e-29
Time:                        13:13:18   Log-Likelihood:                -527.35
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      46   BIC:                             1070.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.625e+04   1013.373     16.036      0.000    1.42e+04    1.83e+04
x1          1.625e+04   1013.373     16.036      0.000    1.42e+04    1.83e+04
x2          1.625e+04   1013.373     16.036      0.000    1.42e+04    1.83e+04
x3          1163.8663   3379.767      0.344      0.732   -5639.252    7966.985
x4             9.5969   3312.045      0.003      0.998   -6657.204    6676.398
x5             0.8530      0.030     28.226      0.000       0.792       0.914
==============================================================================
Omnibus:                       13.418   Durbin-Watson:                   1.122
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               17.605
Skew:                          -0.907   Prob(JB):                     0.000150
Kurtosis:                       5.271   Cond. No.                     4.01e+21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.34e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""